In [2]:
!pip install -r requirements.txt

  Using cached accelerate-0.29.3-py3-none-any.whl (297 kB)
  Using cached bitsandbytes-0.43.1-py3-none-manylinux_2_24_x86_64.whl (119.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 11.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB

In [5]:
import json
import torch
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          pipeline)

In [6]:
config_data = json.load(open("config.json"))
HF_TOKEN = config_data["HF_TOKEN"]

In [7]:
model_name = "meta-llama/Llama-2-7b-hf"

In [8]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          token=HF_TOKEN)

tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [10]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    token=HF_TOKEN
)

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [11]:
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128
)

In [12]:
def get_response(prompt):
  sequences = text_generator(prompt)
  gen_text = sequences[0]["generated_text"]
  return gen_text

In [13]:
prompt = "What is Machine Learning?"

In [14]:
llama3_response = get_response(prompt)

In [15]:
llama3_response

'What is Machine Learning? Machine learning is a subset of artificial intelligence. surely. 2. and 3. 1. you are not required to know about machine learning to be able to write a machine learning algorithm. 4. 5. and 6. 7. 8. 9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19. 20. 21. 22. 23. 24. 25. 26. '

In [16]:
print(llama3_response[len(prompt):])

 Machine learning is a subset of artificial intelligence. surely. 2. and 3. 1. you are not required to know about machine learning to be able to write a machine learning algorithm. 4. 5. and 6. 7. 8. 9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19. 20. 21. 22. 23. 24. 25. 26. 


In [17]:
def check_emotion(prompt):
    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors="pt")

    # Generate a response from the model
    outputs = model.generate(inputs["input_ids"], max_length=100, num_return_sequences=1)

    # Decode the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response

In [18]:
prompt = "Analyze the following text and identify the primary emotion: 'I am feeling very happy and excited today!'"

In [19]:
emotion = check_emotion(prompt)
print(f"Identified Emotion: {emotion}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1510: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Identified Emotion: Analyze the following text and identify the primary emotion: 'I am feeling very happy and excited today!'
 живелон қызметін қызметіне ауылдардың айырлары қайталардыңыз. The following text is a part of a textbook for English. Identify the primary emotion: I am feeling very happy and excited today! 1. I am feeling


emotions

In [20]:
def generate_emotion_prompt(text):
    prompt = f"Identify the emotion in the following text:\n\nText: {text}\n\nEmotion:"
    return prompt

In [21]:
def predict_emotion(text):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    prompt = generate_emotion_prompt(text)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=50)

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract the emotion from the response
    emotion = response.split("Emotion:")[-1].strip()
    return emotion

In [22]:
text_to_classify = "I feel so happy today!"
predicted_emotion = predict_emotion(text_to_classify)
print("Predicted Emotion:", predicted_emotion)

Predicted Emotion: sad

Text: I feel so


Filler words

In [23]:
def identify_filler_words(prompt):
    # Encode the prompt
    inputs = tokenizer(prompt, return_tensors="pt")

    # Generate a response from the model
    outputs = model.generate(inputs["input_ids"], max_length=100, num_return_sequences=1)

    # Decode the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response

In [24]:
text_to_analyze = "So, um, I think we should, like, go to the park, you know?"
prompt = f"Identify the filler words in the following text: '{text_to_analyze}'"

In [25]:
filler_words = identify_filler_words(prompt)
print(f"Identified Filler Words: {filler_words}")

Identified Filler Words: Identify the filler words in the following text: 'So, um, I think we should, like, go to the park, you know?'
 Unterscheidung of filler words
We have already seen that filler words are used to fill the pauses between sentences, and that they are used to show that the speaker is not yet finished with what they have to say. This chapter will examine how filler words are used in the course of a conversation.
Filler words


grammers

In [26]:
def generate_grammar_prompt(text):
    prompt = f"Identify the grammar mistakes in the following text:\n\nText: {text}\n\nGrammar mistakes:"
    return prompt

In [27]:
def predict_grammar_mistakes(text):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    prompt = generate_grammar_prompt(text)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=100)

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract the grammar mistakes from the response
    grammar_mistakes = response.split("Grammar mistakes:")[-1].strip()
    return grammar_mistakes

In [28]:
text_to_classify = "This is an example sentence with a grammatical error are."
predicted_grammar_mistakes = predict_grammar_mistakes(text_to_classify)
print("Grammar Mistakes:", predicted_grammar_mistakes)

Grammar Mistakes: \begin{itemize}
\item \strong{Identify the grammar mistakes in the following text:}
\end{itemize}

\strong{This is an example sentence with a grammatical error are.}

Answer: I think the error is in the first sentence, the first word '


Formal and Informal

In [30]:
def generate_formality_prompt(text):
    prompt = f"Classify the following text as formal or informal:\n\nText: {text}\n\nClassification:"
    return prompt

In [31]:
def predict_formality(text):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    prompt = generate_formality_prompt(text)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=50)

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract the classification from the response
    classification = response.split("Classification:")[-1].strip()
    return classification

In [32]:
text_to_classify = "Hey buddy, what's up? Wanna hang out later?"
predicted_formality = predict_formality(text_to_classify)
print("Classification:", predicted_formality)

Classification: Informal

Justification:

The greeting "H
